# Linear regression demo

Load packages we will use

In [1]:
# dataframes
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn

# linear regression two ways
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# for choosing covariates to include in model
from patsy import dmatrices 

# Basic python

In [2]:
# basic math 
2+2

4

In [3]:
2*pi

NameError: name 'pi' is not defined

In [ ]:
from numpy import pi
2 * pi

In [ ]:
# lists
a = [1, 2, 3]
a

In [ ]:
a.append(17)
a

In [ ]:
# python uses 0-based indexing 
a[0]

In [ ]:
# dictionaries 
d = {'a': 1, 'b': 2, 'c': 3}
d

In [ ]:
d['b']

In [ ]:
d['d']

In [ ]:
d['d'] = 4
d['d']

In [ ]:
# for loops
for i in range(10):
    print("hello", i)

In [ ]:
# functions, if statements
def fibonacci(n):
    if n==0:
        return 1
    else:
        return n*fibonacci(n-1)
    
fibonacci(5)

# Jupyter workflow tips

* run a cell with Shift-Enter
* Jupyter displays value of last expression in cell 
* open a new cell below the current one (Alt-Enter) to see values of variables, test assumptions
* you can reorder cells using the arrows in the toolbar, or by copy-pasting cells up or down

# Electricity usage example

In [ ]:
usage = pd.read_csv('elec_usage.txt')
usage.head()

In [ ]:
usage.hist()

In [ ]:
usage.plot.scatter(x='temperature', y='usage')

In [ ]:
seaborn.pairplot(usage)

In [ ]:
# add two more predictors: nonlinear functions of original variables
usage['tempsqr'] = usage['temperature']^2
usage['yearcts'] = usage['year'] + (usage['month']-1)/12
usage.head()

In [ ]:
variables = list(usage.columns)
variables

In [ ]:
features = variables[0:2] + variables[3:]
features

In [ ]:
target = variables[2]
target

In [ ]:
X = usage[features]
Y = usage[target]
X.head()

In [ ]:
# more concise way of collecting data
Y, X = dmatrices('usage ~ 0 + temperature + np.power(temperature, 2) + month + year + yearcts', data=usage, return_type='dataframe')
X.head()

modifications:
    * add an intercept to the model
    * add other nonlinear terms

In [ ]:
# fit a least squares model
model = sm.OLS(Y, X).fit()
Yhat = model.predict(X) # make the predictions by the model

# Print out the statistics
print(model.summary())

In [ ]:
# Plot outputs
plt.scatter(Y, Yhat,  color='black')
plt.xlabel("true outcome Y")
plt.ylabel("predicted outcome Yhat")

plt.show()

In [ ]:
# plot observed vs predicted usage as a function of temperature
p = usage.plot.scatter(x='temperature', y='usage', label='observed')
p.scatter(x=usage['temperature'], y=Yhat, color='r', marker="+", label='predicted')
plt.legend()

In [ ]:
# let's make a function to plot predictions against truth
def plot_fit(Y, Yhat):
    plt.scatter(Y, Yhat,  color='black')
    plt.xlabel("true outcome Y")
    plt.ylabel("predicted outcome Yhat")

    plt.show()
    
plot_fit(Y, Yhat)

In [ ]:
# we could even make a function to help us visually search for a good model 

def assess_model_quality(data, formula):
    Y, X = dmatrices(formula, data=data, return_type='dataframe')
    model = sm.OLS(Y, X).fit()
    Yhat = model.predict(X)
    plot_fit(Y, Yhat)
    return model.summary()
    
assess_model_quality(usage, 'usage ~ 1 + temperature')

# Now let's fit the same model, using sklearn

In [ ]:
lm = LinearRegression()
model = lm.fit(X,Y)
model

In [ ]:
Yhat = lm.predict(X)
Yhat[:5]

In [ ]:
# how well does the model fit? 
lm.score(X,Y) # R^2 score

In [ ]:
# what about the coefficients?
lm.coef_

In [ ]:
# and the intercept β_0
lm.intercept_

In [ ]:
# and visualize the fit 
plot_fit(Y, Yhat)

# Test assumptions: normality, independence, homoskedasticity

In [ ]:
resid = model.resid

In [ ]:
# test mutual independence 
plt.scatter(resid[:-1], resid[1:])